In [2]:
import os
os.chdir(r"C:\Users\Anil\OneDrive\Desktop\nlp_NER")
os.getcwd()

'C:\\Users\\Anil\\OneDrive\\Desktop\\nlp_NER'

In [10]:
import requests
import json

from google.cloud import language
from google.oauth2 import service_account
from google.cloud.language import enums
from google.cloud.language import types
import numpy as np
import pandas as pd


from google.cloud import language_v1
from google.cloud.language_v1 import enums

client = language_v1.LanguageServiceClient.from_service_account_json('services.json')


def sample_analyze_entities(text_content):
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    encoding_type = enums.EncodingType.UTF8
    response = client.analyze_entities(document, encoding_type=encoding_type)
    a = []
    for entity in response.entities:
        a.append(entity.name)
    return(a)
        
 #' Splitting the ReviewText
#' Loading the Data into Python
DataFrame=pd.read_excel(r"C:\Users\Anil\OneDrive\Desktop\nlp_NER\DataFrame.xlsx",encoding = 'unicode_escape')

#' Removing the duplicates from the dataframe
DataFrame=DataFrame.drop_duplicates("Review_Text",keep="last")

#Removing the NA's from dataframe
Initial_DataFrame=DataFrame[pd.notnull(DataFrame['Review_Text'])]

#' Convert review into lower case
Initial_DataFrame['Review_Text']=Initial_DataFrame['Review_Text'].str.lower()
New_DataFrame=pd.DataFrame(Initial_DataFrame.Review_Text.str.split('.').tolist(),index=Initial_DataFrame['Review_ID']).stack()
New_DataFrame=New_DataFrame.to_frame()
New_DataFrame['Review_SID'] = [str(x) +'_' + str(y) for x,y in New_DataFrame.index]
New_DataFrame=New_DataFrame.reset_index([0,"Review_ID"])
New_DataFrame.columns=['Review_ID',"Review_Text_Breaked","Review_SID"]
New_DataFrame["Review_Text_Breaked"]=New_DataFrame["Review_Text_Breaked"].str.strip()
New_DataFrame["Review_Text_Breaked"].replace('',np.nan,inplace=True)
New_DataFrame=New_DataFrame[['Review_ID',"Review_SID","Review_Text_Breaked"]]

#' Deleting the NA's from the data frame & resetting the index for the iteraion
New_DataFrame=New_DataFrame.dropna()
New_DataFrame.reset_index(inplace=True)
del(New_DataFrame['index'])

#' PreProcessinr_DataFrame
PreProcessing_DataFrame=pd.merge(New_DataFrame,Initial_DataFrame,on="Review_ID")
PreProcessing_DataFrame=PreProcessing_DataFrame[['Review_ID','Review_Text','Review_SID','Review_Text_Breaked']]

#' Preprocessing Data:Removing the pounctuation
import string
from string import punctuation
string.punctuation
def remove_punct(text):
    text_nopunct="".join([char for char in text if char not in punctuation])
    return(text_nopunct)
PreProcessing_DataFrame['Review_Text_Split']=PreProcessing_DataFrame['Review_Text_Breaked'].apply(lambda x:remove_punct(x))




C:\Users\Anil\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
df = PreProcessing_DataFrame[['Review_ID','Review_Text_Split']]
df['GoogleEntities'] = df.Review_Text_Split.apply(lambda x: sample_analyze_entities(x))
df.to_csv("allreviews.csv")

C:\Users\Anil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
df


,Review_ID,Review_Text_Split,GoogleEntities
0,2301_1_Q1,location and friendly professional staff,"[location, staff]"
1,2301_1_Q1,valet service was a plus,[valet service]
2,2301_2_Q1,it was great the only thing that slightly bugg...,"[thing, water container, elevator, hall]"
3,2301_2_Q1,i work across the lot from a marriott and they...,"[lot, marriott, courtesy, guests]"
4,2301_3_Q1,clean and very friendly staff,[staff]
5,2301_4_Q1,room was clean and comfortable,[room]
6,2301_4_Q1,staff were friendly and helpful especially che...,"[staff, staff]"
7,2301_5_Q1,wheelchair accessible room was done right,[room]
8,2301_5_Q1,location is great,[location]
9,2301_6_Q1,location location location,[location location location]
